In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
from glob import glob 
from pathlib import Path 
import os 
from fastcore.xtras import load_pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use(["science", "nature"])

import matplotx

In [3]:
all_out = glob('out/**/*.pkl')

In [4]:
len(all_out)

14

In [5]:
all_res = [load_pickle(p) for p in all_out]

In [8]:
all_compiled_res = []
all_compiled_baseline_res = [] 

for res in all_res:
    compiled_res = {}
    compiled_baseline_res = {}
    compiled_res['accuracy'] = res['accuracy']
    compiled_res['f1_macro'] = res['f1_macro']
    compiled_res['f1_micro'] = res['f1_micro']
    compiled_res['kappa'] = res['kappa']
    compiled_res['train_size'] = res['train_size']

    try:
        compiled_baseline_res['accuracy'] = res['baseline']['accuracy']
        compiled_baseline_res['f1_macro'] = res['baseline']['f1_macro']
        compiled_baseline_res['f1_micro'] = res['baseline']['f1_micro']
        compiled_baseline_res['kappa'] = res['baseline']['kappa']
        compiled_baseline_res['train_size'] = res['train_size']
    except KeyError:
        compiled_baseline_res['accuracy'] = None
        compiled_baseline_res['f1_macro'] = None
        compiled_baseline_res['f1_micro'] = None
        compiled_baseline_res['kappa'] = None
        compiled_baseline_res['train_size'] = res['train_size']
        
    all_compiled_res.append(compiled_res)

    if compiled_baseline_res['accuracy'] is not None:
        if compiled_baseline_res['accuracy'] < 1:
            all_compiled_baseline_res.append(compiled_baseline_res)



In [9]:
all_compiled_res = pd.DataFrame(all_compiled_res)
all_compiled_baseline_res = pd.DataFrame(all_compiled_baseline_res)

In [10]:
all_compiled_res.groupby('train_size').agg(['mean', 'std'])

accuracy            f1_macro            f1_micro            \
                mean       std      mean       std      mean       std   
train_size                                                               
10          0.541845  0.028140  0.499887  0.025530  0.541845  0.028140   
50          0.691542  0.019795  0.690972  0.020133  0.691542  0.019795   
100         0.765585  0.002019  0.765341  0.001695  0.765585  0.002019   
200         0.806926  0.001648  0.806912  0.001628  0.806926  0.001648   
500         0.846641       NaN  0.846576       NaN  0.846641       NaN   
1000        0.849528       NaN  0.849525       NaN  0.849528       NaN   

               kappa            
                mean       std  
train_size                      
10          0.083211  0.058483  
50          0.383114  0.039507  
100         0.531249  0.004183  
200         0.613880  0.003319  
500         0.693335       NaN  
1000        0.699051       NaN

In [11]:
all_compiled_baseline_res.groupby('train_size').agg(['mean', 'std'])

accuracy      f1_macro      f1_micro         kappa    
                mean std      mean std      mean std      mean std
train_size                                                        
10          0.717369 NaN  0.715764 NaN  0.717369 NaN  0.435075 NaN
50          0.793677 NaN  0.793078 NaN  0.793677 NaN  0.587187 NaN
100         0.828251 NaN  0.828229 NaN  0.828251 NaN  0.656477 NaN
200         0.829273 NaN  0.829258 NaN  0.829273 NaN  0.658579 NaN